In [11]:
import keras
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
from saab import Saab
from cwSaab import cwSaab
from pixelhop2 import Pixelhop2
from cross_entropy import Cross_Entropy
from lag import LAG
from skimage.util import view_as_windows
import skimage.measure
import os
import time
from sklearn.model_selection import train_test_split


#taking image dataset
(train_data,train_label), (test_data,test_label)= cifar10.load_data()
print(train_data.shape,train_label.shape, test_data.shape, test_label.shape)

# Preprocessing
train_data=( train_data.astype('float') )
test_data=( test_data.astype('float') )
train_data /= 255.0
test_data /= 255.0

# Normalizing data
# Calculating mean image 
mean_image = np.mean(train_data, axis=0) 
# Calculating standard deviation 
std = np.std(train_data, axis=0)  

# Subtracting calculated mean image from datasets
train_data -= mean_image
test_data -= mean_image

# Dividing then every dataset by standard deviation
train_data /= std
test_data /= std

X_train, X_test, Y_train, Y_test = train_test_split(train_data, train_label, test_size=0.8, random_state=0, stratify= train_label)
print(X_train.shape)


del Y_train
del Y_test

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(10000, 32, 32, 3)


In [0]:
start=time.time()
def Shrink(X, shrinkArg):
  win = shrinkArg['win']
  stride = shrinkArg['stride']
  ch = X.shape[-1]
  X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
  X = X.reshape(X.shape[0], X.shape[1], X.shape[2],-1)
  return skimage.measure.block_reduce(X, (1,2,2,1), np.max)


# example callback function for how to concate features from different hops
def Concat(X, concatArg):
  return X

SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'useDC':True, 'batch':None, 'cw':False}, 
              {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True},
              {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True}]
#               {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True}]
shrinkArgs = [{'func': Shrink, 'win':5, 'stride': 1}, 
               {'func': Shrink, 'win':5, 'stride': 1},
                {'func': Shrink, 'win':5, 'stride': 1}]
#                {'func': Shrink, 'win':3, 'stride': 1}]
concatArg = {'func':Concat}


print(" -----> depth=3")
p2 = Pixelhop2(depth=3, TH1=0.0003, TH2=0.00003, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
p2.fit(X_train)
print("------- DONE -------\n")


output1 = p2.transform(train_data[:5000,:,:])
output2 = p2.transform(train_data[5000:10000,:,:])
output3 = p2.transform(train_data[10000:15000,:,:])
output4 = p2.transform(train_data[15000:20000,:,:])
output5 = p2.transform(train_data[20000:25000,:,:])
output6 = p2.transform(train_data[25000:30000,:,:])
output7 = p2.transform(train_data[30000:35000,:,:])
output8 = p2.transform(train_data[35000:40000,:,:])
output9 = p2.transform(train_data[40000:45000,:,:])
output10 = p2.transform(train_data[45000:50000,:,:])

output_1= np.concatenate((output1[0], output2[0], output3[0],output4[0], output5[0], output6[0],output7[0], output8[0], output9[0], output10[0]))
output_2= np.concatenate((output1[1], output2[1], output3[1],output4[1], output5[1], output6[1],output7[1], output8[1], output9[1], output10[1]))
output_3= np.concatenate((output1[2], output2[2], output3[2],output4[2], output5[2], output6[2],output7[2], output8[2], output9[2], output10[2]))
#output_4= np.concatenate((output1[3], output2[3], output3[3],output4[3], output5[3], output6[3],output7[3], output8[3], output9[3], output10[3]))


print(output_1.shape, output_2.shape, output_3.shape)
print("------- DONE -------\n")


# Save module 1 all outputs
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name1 = 'Projecthw6_module1_output_1'
model_name2 = 'Projecthw6_module1_output_2'
model_name3 = 'Projecthw6_module1_output_3'
#model_name4 = 'Projecthw6_module1_output_4'


if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path1 = os.path.join(save_dir, model_name1)
model_path2 = os.path.join(save_dir, model_name2)
model_path3 = os.path.join(save_dir, model_name3)
#model_path4 = os.path.join(save_dir, model_name4)


np.save(model_path1,output_1)
np.save(model_path2,output_2)
np.save(model_path3,output_3)
#np.save(model_path4,output_4)
print('Saved trained model at %s ' % model_path1)
print('Saved trained model at %s ' % model_path2)
print('Saved trained model at %s ' % model_path3)
#print('Saved trained model at %s ' % model_path4)


end=time.time()
print('Time', end-start)


 -----> depth=3
pixelhop2 fit


/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)


------- DONE -------

pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
(50000, 14, 14, 57) (50000, 5, 5, 404) (50000, 1, 1, 1316)
------- DONE -------

Saved trained model at /content/saved_models/Projecthw6_module1_output_1 
Saved trained model at /content/saved_models/Projecthw6_module1_output_2 
Saved trained model at /content/saved_models/Projecthw6_module1_output_3 
Time 420.2208607196808


In [0]:
#testing data 
#part 2

output_test=p2.transform(test_data)
print(output_test[0].shape, output_test[1].shape, output_test[2].shape)
print("------- DONE -------\n")

pixelhop2 transform
(10000, 14, 14, 57) (10000, 5, 5, 404) (10000, 1, 1, 1316)
------- DONE -------



In [0]:
X_ori_h1_test = output_test[0].reshape((len(output_test[0]), -1))
X_ori_h2_test = output_test[1].reshape((len(output_test[1]), -1))
X_ori_h3_test = output_test[2].reshape((len(output_test[2]), -1))
#X_ori_h4_test = output_test[3].reshape((len(output_test[3]), -1))

print(X_ori_h1_test.shape)
print(X_ori_h2_test.shape)
print(X_ori_h3_test.shape)
#print(X_ori_h4_test.shape)


(10000, 11172)
(10000, 10100)
(10000, 1316)


In [0]:
start=time.time()
print("Module 2: HOP 1")
X_ori_h1 = output_1.reshape((len(output_1), -1))
ce = Cross_Entropy(num_class=10, num_bin=5)
feat_ce_h1 = np.zeros(X_ori_h1.shape[-1])
for k in range(X_ori_h1.shape[-1]):
  feat_ce_h1[k] = ce.compute(X_ori_h1[:,k].reshape(-1,1), train_label)
  #print(" --> KMeans ce: %s"%str(feat_ce[k]))
print(" --> KMeans ce: %s"%str(feat_ce_h1[k]))
print("------- DONE -------\n")

# Saving model 2 op1
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'pmodule2_feat_ce_h1'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

np.save(model_path,feat_ce_h1)
print('Saved trained model at %s ' % model_path)
end=time.time()
print(end-start)

Module 2: HOP 1
 --> KMeans ce: 0.3249477402191003
------- DONE -------

Saved trained model at /content/saved_models/pmodule2_feat_ce_h1 
501.6690580844879


In [0]:
start=time.time()
print("Module 2: HOP 2")
X_ori_h2 = output_2.reshape((len(output_2), -1))
ce = Cross_Entropy(num_class=10, num_bin=5)
feat_ce_h2 = np.zeros(X_ori_h2.shape[-1])
for k in range(X_ori_h2.shape[-1]):
  feat_ce_h2[k] = ce.compute(X_ori_h2[:,k].reshape(-1,1), train_label)
  #print(" --> KMeans ce: %s"%str(feat_ce[k]))
print(" --> KMeans ce: %s"%str(feat_ce_h2[k]))
print("------- DONE -------\n")

# Saving model 2 op2
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'pmodule2_feat_ce_h2'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

np.save(model_path,feat_ce_h2)
print('Saved trained model at %s ' % model_path)
end=time.time()
print(end-start)

Module 2: HOP 2
 --> KMeans ce: 0.28494741450773864
------- DONE -------

Saved trained model at /content/saved_models/pmodule2_feat_ce_h2 
445.79697704315186


In [0]:
start=time.time()
print("Module 2: HOP 3")
X_ori_h3 = output_3.reshape((len(output_3), -1))
ce = Cross_Entropy(num_class=10, num_bin=5)
feat_ce_h3 = np.zeros(X_ori_h3.shape[-1])
for k in range(X_ori_h3.shape[-1]):
  feat_ce_h3[k] = ce.compute(X_ori_h3[:,k].reshape(-1,1), train_label)
  #print(" --> KMeans ce: %s"%str(feat_ce[k]))
print(" --> KMeans ce: %s"%str(feat_ce_h3[k]))
print("------- DONE -------\n")

# Saving model 2 op3
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'pmodule2_feat_ce_h3'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

np.save(model_path,feat_ce_h3)
print('Saved trained model at %s ' % model_path)
end=time.time()
print(end-start)

Module 2: HOP 3
 --> KMeans ce: 0.3249477402191002
------- DONE -------

Saved trained model at /content/saved_models/pmodule2_feat_ce_h3 
55.41390037536621


In [0]:
#Taking top 50% from each cross entropy unit
X_ori_h1 = output_1.reshape((len(output_1), -1))
X_ori_h2 = output_2.reshape((len(output_2), -1))
X_ori_h3 = output_3.reshape((len(output_3), -1))
#X_ori_h4 = output_4.reshape((len(output_4), -1))



a1 = np.argsort(feat_ce_h1, axis=-1, kind= None, order= None)
print("Shape %s"%str(a1.shape))
b1=int(len(a1)/2)
print(b1)
a1=a1[0:1000]
print("New Shape %s"%str(a1.shape))
new_feat_h1=X_ori_h1[:,a1]
print(new_feat_h1.shape)


a2 = np.argsort(feat_ce_h2, axis=-1, kind= None, order= None)
print("Shape %s"%str(a2.shape))
b2=int(len(a2)/2)
print(b2)
a2=a2[0:1000]
print("New Shape %s"%str(a2.shape))
new_feat_h2=X_ori_h2[:,a2]
print(new_feat_h2.shape)


a3 = np.argsort(feat_ce_h3, axis=-1, kind= None, order= None)
print("Shape %s"%str(a3.shape))
b3=int(len(a3)/2)
print(b3)
a3=a3[0:1000]
print("New Shape %s"%str(a3.shape))
new_feat_h3=X_ori_h3[:,a3]
print(new_feat_h3.shape)



#a4 = np.argsort(feat_ce_h4, axis=-1, kind= None, order= None)
#print("Shape %s"%str(a4.shape))
#b4=int(len(a4)/2)
#print(b4)
#a4=a4[0:b4]
#print("New Shape %s"%str(a4.shape))
#new_feat_h4=X_ori_h3[:,a4]

# Saving model 2 op1 new
model_name = 'pmodule2_feat_ce_h1_top50'
model_path = os.path.join(save_dir, model_name)
np.save(model_path,new_feat_h1)
print('Saved trained model at %s ' % model_path)

# Saving model 2 op2 new
model_name = 'pmodule2_feat_ce_h2_top50'
model_path = os.path.join(save_dir, model_name)
np.save(model_path,new_feat_h2)
print('Saved trained model at %s ' % model_path)

# Saving model 2 op3 new
model_name = 'pmodule2_feat_ce_h3_top50'
model_path = os.path.join(save_dir, model_name)
np.save(model_path,new_feat_h3)
print('Saved trained model at %s ' % model_path)

# Saving model 2 op4 new
#model_name = 'pmodule2_feat_ce_h4_top50'
#model_path = os.path.join(save_dir, model_name)
#np.save(model_path,new_feat_h4)
#print('Saved trained model at %s ' % model_path)

Shape (11172,)
5586
New Shape (1000,)
(50000, 1000)
Shape (10100,)
5050
New Shape (1000,)
(50000, 1000)
Shape (1316,)
658
New Shape (1000,)
(50000, 1000)
Saved trained model at /content/saved_models/pmodule2_feat_ce_h1_top50 
Saved trained model at /content/saved_models/pmodule2_feat_ce_h2_top50 
Saved trained model at /content/saved_models/pmodule2_feat_ce_h3_top50 


In [0]:
new_feat_h1_test=X_ori_h1_test[:,a1]
new_feat_h2_test=X_ori_h2_test[:,a2]
new_feat_h3_test=X_ori_h3_test[:,a3]
#new_feat_h4_test=X_ori_h4_test[:,a4]
print(new_feat_h1_test.shape)
print(new_feat_h2_test.shape)
print(new_feat_h3_test.shape)

print('>----Done----')

(10000, 1000)
(10000, 1000)
(10000, 1000)
>----Done----


In [0]:
from llsr import LLSR as myLLSR
from sklearn.ensemble import RandomForestClassifier
lag = LAG(encode='distance', num_clusters=[5,5,5,5,5,5,5,5,5,5], alpha=10, learner=myLLSR(onehot=False))  

lag.fit(new_feat_h1, train_label)
X_train_trans1 = lag.transform(new_feat_h1)
print(X_train_trans1.shape)
X_train_trans1_test = lag.transform(new_feat_h1_test)
print('Done')

lag.fit(new_feat_h2, train_label)
X_train_trans2 = lag.transform(new_feat_h2)
print(X_train_trans2.shape)
X_train_trans2_test = lag.transform(new_feat_h2_test)
print('Done')

lag.fit(new_feat_h3, train_label)
X_train_trans3 = lag.transform(new_feat_h3)
print(X_train_trans3.shape)
X_train_trans3_test = lag.transform(new_feat_h3_test)
print('Done')

#lag.fit(new_feat_h4, train_label)
#X_train_trans4 = lag.transform(new_feat_h4)

X_train_trans = np.concatenate((X_train_trans1,X_train_trans2,X_train_trans3), axis=1)
print(X_train_trans.shape)


# Saving model 2 op3 new
model_name = 'pconcat_4lag_train'
model_path = os.path.join(save_dir, model_name)
np.save(model_path,X_train_trans)
print('Saved trained model at %s ' % model_path)

#X_train_predprob = lag.predict_proba(X_train_trans)
#print(" --> train acc: %s"%str(lag.score(X_train_predprob, train_label)))
#print(" --> test acc.: %s"%str(lag.score(X_test, y_test)))
print("------- DONE -------\n")

X_test_trans = np.concatenate((X_train_trans1_test,X_train_trans2_test,X_train_trans3_test), axis=1)

# Saving model 2 op3 new
model_name = 'pconcat_4lag_test'
model_path = os.path.join(save_dir, model_name)
np.save(model_path,X_test_trans)
print('Saved trained model at %s ' % model_path)

(50000, 50)
Done
(50000, 50)
Done
(50000, 50)
Done
(50000, 150)
Saved trained model at /content/saved_models/pconcat_4lag_train 
------- DONE -------

Saved trained model at /content/saved_models/pconcat_4lag_test 


In [18]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import time

start=time.time()

clf1 = RandomForestClassifier(n_estimators=500)
clf1.fit(X_train_trans,train_label)
X_train_pred = clf1.predict(X_train_trans)

store=accuracy_score(train_label,X_train_pred)
print(" --> Train acc: ",store)
end=time.time()
print('Total time', end- start)
print("------- DONE -------\n")


# Saving  classifier
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model_name = 'classifier1'
model_path = os.path.join(save_dir, model_name)
np.save(model_path,clf1)
print('Saved trained model at %s ' % model_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


 --> Train acc:  1.0
Total time 561.1627118587494
------- DONE -------

Saved trained model at /content/saved_models/classifier1 


In [19]:
#prediction for test

X_test_pred = clf1.predict(X_test_trans)

store_test=accuracy_score(test_label,X_test_pred)
print(" --> Testing Accuracy: ",store_test)
print("------- DONE -------\n")

 --> Testing Accuracy:  0.6316
------- DONE -------

